# Setup
1. install python (3.8) and git
2. install virtualenv e.g. with `<path to python> -m to pip install virtualenv`
3. create a virtual environment that uses python 3.8 using virtualenv e.g. with `<path to python> -m virtualenv "<path to virtual environment>"`
4. activate the virtual environment e.g. on windows with `.\<pathto env>\Scripts\activate`
5. install vscode and python extension
6. git clone brainlit (https://github.com/neurodata/brainlit) and neuroglancer (https://github.com/google/neuroglancer)
7. install brainlit from source e.g. `pip install -e .` in brainlit folder 

In [1]:
from brainlit.utils.write import czi_to_zarr, zarr_to_omezarr
import zarr
from cloudvolume import CloudVolume

# Convert czi to zarr

In [ ]:
czi_path = "/cis/project/sriram/Sriram/SS IUE 175 SNOVA RFP single channel AdipoClear Brain 1 full Image 3.czi"
out_dir = "/cis/project/sriram/ng_data/sriram-adipo-brain1-im3/"

zarr_paths = czi_to_zarr(czi_path=czi_path, out_dir=out_dir)

# Convert zarr to ome-zarr

In [ ]:
out_path = "/cis/project/sriram/ng_data/sriram-adipo-brain1-im3/fg_ome.zarr"

zarr_to_omezarr(zarr_path=zarr_paths[0], out_path=out_path)

# View ome-zarr on neuroglancer

1. Use https://github.com/google/neuroglancer/blob/master/cors_webserver.py to serve the data e.g. `python cors_webserver.py -d "/cis/project/sriram/ng_data/" -p 9010`
2. Launch neuroglancer app e.g. https://neuroglancer-demo.appspot.com
3. Add layer to neuroglancer app e.g. zarr://http://127.0.0.1:9010/sriram-adipo-brain1-im3/fg_ome.zarr

# Add trace layer

In [ ]:
z = zarr.open(out_dir + "fg.zarr")
volume_size = z.shape
chunk_size = z.chunks
resolution = [1, 1, 1]

outpath = (
    "precomputed://file:///cis/project/sriram/ng_data/sriram-adipo-brain1-im3/traces"
)

info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="segmentation",
    data_type="uint16",  # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, zfpc, compresso
    encoding="raw",
    resolution=resolution,  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=chunk_size,  # units are voxels
    volume_size=volume_size,  # e.g. a cubic millimeter dataset
    skeletons="skeletons",
)
vol = CloudVolume(outpath, info=info, compress=False)
vol.commit_info()
vol.skeleton.meta.commit_info()